# Segmenting and Clustering Neighbourhoods in Toronto Part 1

## 1. Import Libraries

In [7]:
import pandas as pd
import requests
!pip install beautifulsoup4
!pip install lxml
from bs4 import BeautifulSoup
import lxml
print('Finished')

     |████████████████████████████████| 112kB 6.0MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 5.6MB/s eta 0:00:01     |██████████▋                     | 1.9MB 5.6MB/s eta 0:00:01
Finished


## 2. Scrape Table from Wikipedia and Convert to a Dataframe

In [8]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
toronto_df=pd.DataFrame(df[0])
toronto_df.head()


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


## 3.a) Remove Rows with Borough 'Not Assigned'

In [9]:
toronto_df = toronto_df.drop(toronto_df[toronto_df.Borough =="Not assigned"].index)

In [10]:
toronto_df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


## 3.b) List Neighbourhoods with a Comma for Separation

In [11]:
toronto_df['Neighborhood']= toronto_df['Neighborhood'].str.replace('/',',') #replace / with ,
toronto_df=toronto_df.reset_index() #reset the index so that it starts at 0
toronto_df.head()

,index,Postal code,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,5,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


## 3.c) Replace N/A Neighbourhoods with Borough Name

In [12]:
toronto_df['Neighborhood']=toronto_df['Neighborhood'].replace('NaN', toronto_df['Borough'])

In [13]:
toronto_df.head(12)

,index,Postal code,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,5,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,8,M9A,Etobicoke,Islington Avenue
6,9,M1B,Scarborough,"Malvern , Rouge"
7,11,M3B,North York,Don Mills
8,12,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [15]:
toronto_df.shape #find number of rows of dataframe

(103, 4)

# Part 2

## 1. Get Latitude and Longitude Dataframe For Each Postal Code

In [32]:
#read lat long CSV into a dataframe
Lat_long=pd.read_csv('Geospatial_Coordinates.csv')
Lat_long.head()



,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## 2.Create lists to of coordinates that correspond with toronto_df

In [60]:
#create empty lists for latitude and longitude
latitude=[]
longitude=[]


In [61]:
#for each column in the toronto_df dataframe find the corresponding coordinates in the lat_long dataframe
for i in range(0,toronto_df.shape[0]):
    postal_code1=toronto_df.iloc[i,1]
    for j in range (0,Lat_long.shape[0]):
        postal_code2=Lat_long.iloc[j,0]
        if postal_code1==postal_code2:
            index=j
            latitude.append(Lat_long.iloc[j,1])
            longitude.append(Lat_long.iloc[j,2])

    


## 3. Add latitude and longitude lists to toronto_df

In [68]:
toronto_df['Latitude']=latitude #add latitude column
toronto_df['Longitude']=longitude #add longitude column

In [71]:
toronto_df.head(12)

,index,Postal code,Borough,Neighborhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.753259,-79.329656
1,3,M4A,North York,Victoria Village,43.725882,-79.315572
2,4,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,5,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,8,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,9,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,11,M3B,North York,Don Mills,43.745906,-79.352188
8,12,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
